# **Projeto Voluntário de Ciência de Dados**



> Autores: Eduardo Magallhães, 
Olivan Rodriguez(ponha seu nome aqui)

> Versão: v0.1

## MVP inicial do projeto



> **1. Objetivos**


       *   Montar uma base única dos dados de localização das barragens e municípios próximos
       *   Realizar análise exploratória simples, com foco em sumarizar estatísticas
       *   Criar um data-viz baseado em geolocalização



> **2. Fontes de dados base**: ANA (Agência Nacional das Águas), Geolocalização dos municípios, ...

> **3. Restrições**: Foco apenas nas barragens de mineração (já serão mais de 600!)

> **4. Output**: Inicial, a ideia será mostrar a localização das barragens com maior risco/não inspecionadas e sua distância do(s) município(s).


# Papéis no projeto:









>**Data Engineer**


          1. Consolidar a(s) base(s) em um formato csv que possa ser lido pelo Pandas - 
          Voluntário(s):Edu Magalhães, Olivan Rodriguez.
          
          2. Formatar as geolocalizações (colocar as latitudes e longitudes no formato consumível pelo Kepler)
          Voluntário(s): Márcio Santos

> **Cientista de Dados (Tech)**

          1. Análise exploratória (sumário estatístico) que poderá ser usado pra ter insights ou ajudar na visualização/clusterização geolocalizada
          Voluntário(s):

>**Cientista de Dados (Business)**

          1. Organização da iniciativa
          Voluntário(s):Edu Magalhães, 
          
          2. Data-viz no Kepler ou ferramenta/biblioteca de geolocalização
          Voluntário(s):
          


# Prazos (para não ficarmos sem meta alguma, rsrs!)



1.   **Domingo, 03/fev** MVP inicial com pelo menos as barragens de Minas geolocalizadas em alguma ferramenta de data-viz






---



# 0.0 - Set up

In [0]:
!ls
!pip install -q pydrive

adc.json  database_versao_01_fonteANM_23_01_2019.csv  sample_data


In [0]:
from google.colab import auth
auth.authenticate_user()

from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
myfile = drive.CreateFile({'id': '1SdzbEMB6GXd2yaF-4KuEyMtB1JBtVsgI'})
myfile.GetContentFile('database_versao_01_fonteANM_23_01_2019.csv')

In [0]:
#Imporatando as bibliotecas
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
%matplotlib inline
import calendar


# 1.0 Carregando os dados

> **Links úteis das princiapis fontes de dados utilizadas:**


>1.   Arquivos das barragens no site do antigo **DNPM** (Departamento Nacional de Produção Mineral, agora **ANM**-Agência Nacional de Mineração) - http://www.anm.gov.br/assuntos/barragens/arquivos-barragens
>> Nesse emaranhado de links, utilizaremos apenas o arquivo pdf do link http://www.anm.gov.br/assuntos/barragens/arquivos-barragens/CADASTRO%20NACIONAL%20DE%20BARRAGENS_2016%20_FINAL%2006-01-2017.pdf/view
>>Tal arquivo nos ajudará a entender as colunas de classificação já geradas, porém, não deve ser muito mais útil que isso por estar em pdf (dificuldade de extrair os dados)

>2.   Inventário de barragens no site da **FEAM**, que é a Fundação Estadual do Meio Ambiente e, por isso, teremos apenas os dados das barragens de MG nesse site/repositório: http://www.feam.br/monitoramento/gestao-de-barragens
>> Aqui, vamos utilizar o link http://www.feam.br/images/stories/2018/BARRAGENS/PLANILHA_DIVULGA%C3%87%C3%83O.xlsx para baixar a planilha com as informações das barragens.
>> Nesse link, temos um relatório bem legal fazendo uma análise exploratória de dados que pode ser melhorada e mais explorada, além de depois podermos cruzar com mais dados, segue o link de referência: http://www.feam.br/images/stories/2018/BARRAGENS/Invent%C3%A1rio_de_Barragens_2017.pdf

>3 Dados extras
>> Dados sismológicos de acesso publico: http://obsis.unb.br/portalsis/ e http://www.sismo.iag.usp.br/eq/latest

>4 Arquivos excel/csv (convertidos do pdf do item 1.)
>>**Excel convertido originalmente**
https://docs.google.com/spreadsheets/d/1Ntj0jr3joIJtnjZQqA9m7Qa-cgya1KiGjjng8mrT_KQ/edit?usp=sharing

>>**CSV a partir do Excel**
https://drive.google.com/file/d/1eb5C6_7LOiLvlhU7KXmvHaQ6Soa7E9EV/view?usp=sharing



## 1.1 Importando a planilha excel do cadastro nacional de barragens Fonte ANM

In [0]:
pd.options.display.float_format = '{:,.2f}'.format

df = pd.read_csv('database_versao_01_fonteANM_23_01_2019.csv', header=0, delimiter=',',encoding = 'utf-8', decimal= ',')

## 1.2 - Primeiros olhares na base de dados e estatística simples/descritiva

In [0]:
df.head()

,NOME_BARRAGEM_MINERACAO,NOME_DO_EMPREENDEDOR,CPF_CNPJ,POSICIONAMENTO,UF,MUNICIPIO,MINERIO_PRINCIPAL,ALTURA_ATUAL_metros,VOLUME_ATUAL_m3,CATEGORIA_DE_RISCO,DANO_POTENCIAL_ASSOCIADO,CLASSE,INSERIDA_NA_PNSB,LATITUDE,LONGITUDE
0,0-1,Mineração Taboca S.a.,34.019.992/0001-10,Sul do Equador,AM,PRESIDENTE FIGUEIREDO,Minério de Estanho Primário,22.00,"27,700,000.00",Baixa,Média,C,Sim,0.733361111111111,-60.1383888888889
1,103 (Cruz),Mineração Taboca S.a.,34.019.992/0001-10,Sul do Equador,AM,PRESIDENTE FIGUEIREDO,Minério de Estanho Primário,10.30,"924,000.00",Baixa,Média,C,Sim,0.785722222222222,-60.1419166666667
2,111 (Índio),Mineração Taboca S.a.,34.019.992/0001-10,Sul do Equador,AM,PRESIDENTE FIGUEIREDO,Minério de Estanho Primário,5.00,"48,873.00",Baixa,Média,C,Sim,0.789666666666667,-60.1442777777778
3,158 (A-1),Mineração Taboca S.a.,34.019.992/0001-10,Sul do Equador,AM,PRESIDENTE FIGUEIREDO,Minério de Estanho Primário,30.00,"53,380,000.00",Baixa,Alta,B,Sim,0.738027777777778,-60.0789166666667
4,161 (A-2),Mineração Taboca S.a.,34.019.992/0001-10,Sul do Equador,AM,PRESIDENTE FIGUEIREDO,Minério de Estanho Primário,22.00,"1,018,054.00",Baixa,Média,C,Sim,0.732286944444444,-60.0689988888889


In [0]:
# Formato da tabela
df.shape

(714, 15)

In [0]:
#Breve estatística descritiva
df.describe(include=['object'])

,NOME_BARRAGEM_MINERACAO,NOME_DO_EMPREENDEDOR,CPF_CNPJ,POSICIONAMENTO,UF,MUNICIPIO,MINERIO_PRINCIPAL,ALTURA_ATUAL_metros,VOLUME_ATUAL_m3,CATEGORIA_DE_RISCO,DANO_POTENCIAL_ASSOCIADO,CLASSE,INSERIDA_NA_PNSB,LATITUDE,LONGITUDE
count,714,714,714,714,714,714,714,714,714,390,390,390,714,714,714
unique,714,233,228,2,20,164,57,207,591,3,3,5,2,704,695
top,Bacia de Acumulação 01,Mineracao Rio do Norte S A,04.932.216/0001-46,Sul do Equador,MG,POCONÉ,Minério de Ferro,5.00,"12,000.00",Baixa,Alta,B,Sim,-19.41065,-47.9556833333333
freq,1,25,25,705,324,31,210,42,10,332,197,196,390,3,6


In [0]:
# Checando dados nulos
# Infelizmente temos dados nulos em algumas colunas... temos que verificar a causa disso.
df.isnull().sum()

NOME_BARRAGEM_MINERACAO       0
NOME_DO_EMPREENDEDOR          0
CPF_CNPJ                      0
POSICIONAMENTO                0
UF                            0
MUNICIPIO                     0
MINERIO_PRINCIPAL             0
ALTURA_ATUAL_metros           0
VOLUME_ATUAL_m3               0
CATEGORIA_DE_RISCO          324
DANO_POTENCIAL_ASSOCIADO    324
CLASSE                      324
INSERIDA_NA_PNSB              0
LATITUDE                      0
LONGITUDE                     0
dtype: int64

# 2.0 - Primeiras explorações

In [0]:
#Eliminando as linhas nulas para ficar mais fácil uma primeira análise
df_nonullvalues = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_nonullvalues.shape

(390, 15)

In [0]:
#Checando novamente a eliminação de valores nulos
df_nonullvalues.isnull().sum()

NOME_BARRAGEM_MINERACAO     0
NOME_DO_EMPREENDEDOR        0
CPF_CNPJ                    0
POSICIONAMENTO              0
UF                          0
MUNICIPIO                   0
MINERIO_PRINCIPAL           0
ALTURA_ATUAL_metros         0
VOLUME_ATUAL_m3             0
CATEGORIA_DE_RISCO          0
DANO_POTENCIAL_ASSOCIADO    0
CLASSE                      0
INSERIDA_NA_PNSB            0
LATITUDE                    0
LONGITUDE                   0
dtype: int64

In [0]:
#... continua...mandem bala quem quiser!